# Capacity Credit Calculator

## **Usage**

1. In Google Colab, press `ctrl+F9`
1. Press `run anyway` if a warning message appear.
1. Wait for the UI to show up (until a loading circle in the bottom of notebook stopped).
1. Upload files by pressing upload button.
1. Press compute to run simulation.
1. Press `ctrl+F7` to restart.

## **Input Format**

### Forced outage rate input format

|No |Unit Name  |Capacity (MW)  |FOR (Outage)   |Status |
|---|-----------|---------------|---------------|-------|
|1  |COAL-1     |50             |0.1            |1      |
|2  |COAL-2     |20             |0.1            |0      |
|3  |COAL-3     |50             |0.2            |1      |
|4  |COAL-4     |30             |0.1            |1      |

Note:

1. status denotes that the generating unit is considered (1) or not (0) in making COPT table.

### Profile input format

|No     |Demand (MW)    |
|-------|---------------|
|1      |150            |
|...    |...            |
|...    |...            |
|8760   |125            |

### VOLL input format

VOLL can use `_` and `,` separator for thousands and `.` for cents, eg: `15_000_000.00`


In [ ]:
# @title
!pip install -qqq itables  # noqa E402

In [ ]:
# @title
import itertools

import numpy as np
import pandas as pd
from ipywidgets import widgets
from itables import show  # noqa E402
from numba import jit

In [ ]:
# @title
COPT_COLUMNS = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                # 'Reversed Cumulative Probability',
                ]
CC_COLUMNS = ['Scenario Name',
              'Penetration Level (%)',
              'VRE Installed Capacity (MWp)',
              'EENS After VRE Installation (MWh)',
              'Delta Load to Equalize VRE (MW)',
              'Capacity Credit (%)',
              ]
NUMPY_ZERO = np.float64(0)

IDX_CAP = 0
IDX_INP = 1
IDX_CMP = 2
# IDX_RCP = 4

# setting numpy print option decimal places
decimal_places = 4
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.4f}'.format)


# copt
# @jit(nopython=True)
def get_copt(
        capacities,
        outage_rates,
        status,
        row_threshold=None,
        min_cum_prob=0.0001,
        row_reduced_to=50):
    # filter only available generator
    # TODO: sort based on frequency of capacities, not capacities size
    generator_list = [[cap, 1 - out]
                      for cap, out, stat
                      in sorted(zip(capacities, outage_rates, status), reverse=True)
                      if stat]

    # make tables for each generator
    # TODO: Tables already support derating, input data should also support it
    # from excel
    tables = [np.array([generator, [0, 1 - generator[1]]])
              for generator in generator_list]

    table = tables[0].copy()  # copy to avoid modifying data
    for table_ in tables[1:]:
        # combine tables
        table = _combine_tables(table, table_)

        # combine duplicate
        table = np.array([[k, sum([x[1] for x in list(g)])]
                          for k, g in itertools.groupby(table, lambda x:x[0])])

        # # merge insignificant
        # try:
        #     if len(table) > row_threshold:
        #         table = _merge_insignificant(table, min_cum_prob)
        # except Exception:  # TypeError
        #     pass

        # # merge resample
        # try:
        #     if len(table) > row_threshold:
        #         table = _merge_resample(table, row_reduced_to)
        # except Exception:  # TypeError
        #     pass

    table = np.hstack((table,
                       # Cumulative Probability
                       np.atleast_2d(np.cumsum(table[::-1, IDX_INP])[::-1]).T,
                       # # Reversed Cumulative Probability
                       # np.atleast_2d(np.cumsum(table[:, IDX_INP])).T,
                       ))

    return table


@jit(nopython=True)
def _combine_tables(table, table_):
    # TODO:
    # Compare between:
    #   1. flatten + transpose
    #   2. np.array([table[:, 0]]).T
    table = np.hstack((
        (
            np.expand_dims(table[:, IDX_CAP], axis=1) + table_[:, IDX_CAP]
        ).reshape(-1, 1),  # sum capacity
        (
            np.expand_dims(table[:, IDX_INP], axis=1) * table_[:, IDX_INP]
        ).reshape(-1, 1),  # multiply probability
    ))

    # sort table
    table = table[(-table[:, IDX_CAP]).argsort(), :]
    return table


# @jit(nopython=True)
def _merge_insignificant(table, min_cum_prob):
    # merge insignificant
    cum_prob = np.cumsum(table[::-1, IDX_INP])[::-1]
    mask_insignificant = cum_prob < min_cum_prob
    if mask_insignificant.sum() > 1:
        # NOTE:
        # ROUNDUP: More accurate but reduce EENS
        # ROUNDDOWN: Not accurate and will amplify EENS
        # ROUNDUPDOWN: Not accurate, increase EENS, but better than ROUNDDOWN

        # # round up approach
        # table = table[~mask_insignificant, :]  # cut
        # table[-1, IDX_INP] = 1 - table[:-1, IDX_INP].sum()  # replace

        # # round down approach
        # table = np.vstack((
        #     table[~mask_insignificant, :],  # cut
        #     np.array([[NUMPY_ZERO, table[mask_insignificant, 1].sum()]])
        # ))

        # # round up down approach
        # removed_cap = table[mask_insignificant, IDX_CAP]
        # removed_inp = table[mask_insignificant, IDX_INP]
        # weight_to_top = (removed_cap
        #                  / table[~mask_insignificant, IDX_CAP][-1])
        # zero_inp = (
        #     table[-1, IDX_INP]  # zero_inp
        #     + np.sum(removed_inp * (1 - weight_to_top))
        # )
        # table = table[~mask_insignificant, :]  # cut
        # table[-1, IDX_INP] = (
        #     table[-1, IDX_INP]  # top_inp
        #     + np.sum(removed_inp * weight_to_top)
        # )
        # table = np.vstack((
        #     table,
        #     np.array([[NUMPY_ZERO, zero_inp]])
        # ))

        pass

    # TODO:
    # Implement resample,
    # useful for big COPT table, triggered only if max table length achieve
    return table


def _merge_resample(table, row_reduced_to):
    group_size = len(table) // row_reduced_to
    reduced_row = group_size * row_reduced_to

    # cut start from index 0 (most likely to be not used by EENS?)
    removed_cap = table[:reduced_row, IDX_CAP].reshape(row_reduced_to, -1)[:, 1:]
    removed_inp = table[:reduced_row, IDX_INP].reshape(row_reduced_to, -1)[:, 1:]
    resampled_cap = np.atleast_2d(
        table[:reduced_row + group_size:group_size, IDX_CAP]).T
    resampled_inp = np.atleast_2d(
        table[:reduced_row + group_size:group_size, IDX_INP]).T

    distance_top = ((resampled_cap[:-1] - removed_cap)
                    / np.diff(resampled_cap, axis=0))
    distance_bot = 1 - distance_top

    resampled_inp[:-1] = (resampled_inp[:-1]  # top, distance_bot
                          + np.atleast_2d(np.sum(removed_inp
                                                 * distance_bot, axis=1)).T)
    resampled_inp[1:] = (resampled_inp[1:]  # bot, distance_top
                         + np.atleast_2d(np.sum(removed_inp
                                                * distance_top, axis=1)).T)

    resampled_tab = np.hstack((resampled_cap, resampled_inp))
    table = np.vstack((resampled_tab, table[reduced_row + 1:]))
    return table


# lolp
@jit(nopython=True)
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    # NOTE: Lowest COPT table < lowest demand cause error
    try:
        idx = np.where(capacity < demand)[0][0]
    except Exception:  # IndexError
        idx = -1
    return cumulative_probability[idx]


# eens
@jit(nopython=True)
def get_eens(capacity, individual_probability, demand):
    """
    format:
        capacity
        individual_probability
    """
    return sum(individual_probability
               * (capacity < demand)
               * (demand - capacity))


@jit(nopython=True)
def find_delta_load(lb, ub, table, net_loads, eens, tol=0.0001):
    # lb_0 = 0
    # ub_0 = max(sun)
    # delta_load_0 = max(sun) / 2
    eens_delta_load = np.inf
    while abs(eens_delta_load - eens) > tol:
        delta_load = (lb + ub) * 0.5
        eens_delta_load = sum([get_eens(table[:, IDX_CAP], table[:, IDX_INP], net_load)
                               for net_load in net_loads + delta_load])
        if eens_delta_load > eens:
            ub = delta_load
        else:
            lb = delta_load
    return delta_load, eens_delta_load

In [ ]:
# @title
# initiate widgets
output = widgets.Output()
for_data_uploader = Uploader(name='FOR Data')
demand_profile_data_uploader = Uploader(name='Demand Profile Data')
vre_profile_data_uploadder = Uploader(name='VRE Profile Data')
voll_widget = widgets.Text(
    value='15_000_000.00',
    description='VOLL: ',
    disabled=False
)
currency_widget = widgets.Dropdown(
    options=['IDR', 'USD'],
    value='IDR',
    description='Currency: ',
    disabled=False,
)
button = widgets.Button(description="Compute!")

In [ ]:
# @title
# button action
def on_button_clicked(b):
    with output:
        # print
        print("Computing...")

        # filter for inputs
        df = for_data_uploader.get_dataframe()
        capacities = df['Capacity (MW)'].tolist()
        outage_rates = df['FOR (Outage)'].tolist()
        status = df['Status'].tolist()

        # # to print input table, use:
        # pd.DataFrame(data={'capacities': capacities,
        #                    'outage_rates': outage_rates,
        #                    'status': status},
        #              index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

        table = get_copt(capacities, outage_rates, status, row_threshold=None)
        df = pd.DataFrame(data=table,
                          columns=COPT_COLUMNS,
                          index=pd.RangeIndex(1, len(table) + 1, 1, name='No'),
                          )
        # # show copt table
        # show(df, "COPT Table")

        # to save COPT in csv
        # CASE_NAME = 'COPT Case.csv'
        # df.to_csv(f'../results/COPT_{CASE_NAME}')

        # TODO: copt downloader

        # filter demand inputs
        demands = demand_profile_data_uploader.get_dataframe()['Demand (MW)'].values

        # EENS original without VRE
        eens = sum([get_eens(table[:, 0], table[:, 1], demand)
                    for demand in demands])
        # print(f'Total EENS (using demand profile): {eens:.4f} MW')

        # capacity credit
        peak_load = demands.max()
        vres = vre_profile_data_uploadder.get_dataframe().set_index('No')
        vres_cf = vres['Capacity Factor (p.u.)'].values

        cc_report = [['BASE SCENARIO', 0, 0, eens, np.nan, np.nan]]
        for column in vres.columns[1:]:
            # print(f'Using demand profile and scenario "{column}":')

            vres_max_cap = peak_load * vres[column].values[0]
            profile = vres_cf * vres_max_cap
            net_loads = demands - profile
            # print(f'    Installed capacity: {vres_max_cap} MWp')

            # EENS on VRE
            eens_vre = sum([get_eens(table[:, 0], table[:, 1], net_load)
                            for net_load in net_loads])

            # print(f'    Total EENS with installed VRE: {eens_vre:.4f} MWh')

            delta_load, eens_delta_load = find_delta_load(
                0, np.max(profile), table, net_loads, eens, tol=0.01)
            capacity_credit = 100 * (delta_load / vres_max_cap)
            # print(f'    Equal to flat load: {delta_load:.4f} MW')
            # print(f'    Capacity credit: {capacity_credit:.4f} %')

            scenario_name = column
            penetration_level = vres[column].values[0] * 100
            cc_report.append([scenario_name, penetration_level,
                              vres_max_cap, eens_vre, delta_load, capacity_credit])

        show(pd.DataFrame(data=cc_report,
                          columns=CC_COLUMNS,
                          index=pd.RangeIndex(1, len(cc_report) + 1, 1, name='No'),
                          ),
             "Capacity Credit Report")


# initiate button
button.on_click(on_button_clicked)

In [ ]:
# @title
# interface
widgets.VBox((for_data_uploader.box,
              demand_profile_data_uploader.box,
              vre_profile_data_uploadder.box,
              button,
              output,
              ))